<a href="https://colab.research.google.com/github/tiara070403/skripsi/blob/main/skripsilagi_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install scikit-learn pandas matplotlib seaborn Sastrawi transformers wordcloud nltk
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from wordcloud import WordCloud
import nltk
nltk.download('punkt')
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from transformers import pipeline
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 1.8 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
df = pd.read_csv('/content/crawling.csv', index_col=0)

In [5]:
df_copy = df.copy()

In [6]:
df = df[['created_at', 'full_text']]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5349 entries, 1858070162362204643 to 1230765438897815552
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   created_at  5349 non-null   object
 1   full_text   5349 non-null   object
dtypes: object(2)
memory usage: 125.4+ KB


In [8]:
# Mengubah teks menjadi lowercase
df['full_text'] = df['full_text'].astype(str).str.lower()
df['full_text'] = df['full_text'].replace('false', pd.NA)
df = df.dropna(subset=['full_text'])

In [9]:
# Menghapus duplikat
initial_duplicates = df.duplicated().sum()
df = df.drop_duplicates(subset=['full_text'])
print(f"Jumlah duplikat awal: {initial_duplicates}")
print(f"Jumlah duplikat setelah drop: {df.duplicated().sum()}")

Jumlah duplikat awal: 4
Jumlah duplikat setelah drop: 0


In [10]:
# Menghapus baris dengan nilai null (setelah dropna subset)
df = df.dropna()
print(f"Jumlah nilai null setelah dropna: \n{df.isnull().sum()}")

Jumlah nilai null setelah dropna: 
created_at    0
full_text     0
dtype: int64


In [11]:
# Case Folding
df['full_text'] = df['full_text'].str.lower()
print("\nDataFrame setelah Case Folding:")
print(df.head())


DataFrame setelah Case Folding:
                                         created_at  \
conversation_id_str                                   
1858070162362204643  Sun Nov 17 08:50:22 +0000 2024   
1846529554657239219  Wed Oct 16 12:32:07 +0000 2024   
1467691301155979267  Wed Oct 09 13:58:44 +0000 2024   
1836014724523458593  Tue Sep 17 12:09:56 +0000 2024   
1835164951125917967  Sun Sep 15 03:53:14 +0000 2024   

                                                             full_text  
conversation_id_str                                                     
1858070162362204643            lucu banget mahasiswa pencinta alam ini  
1846529554657239219  bejat mahasiswa pencinta alam di jambi 'genjot...  
1467691301155979267  @leonita_lestari ada perkoempoelan pentjinta a...  
1836014724523458593  [press release sowan ukm ke ukm unit pandu lin...  
1835164951125917967  [press release sowan ukm ke ukm unit pandu lin...  


In [12]:
# Cleaning
def clean_twitter_text(text):
  text = re.sub(r'@[A-Za-z0-9_]+', '', text) # Menghapus mention
  text = re.sub(r'#\w+', '', text)          # Menghapus hashtag
  text = re.sub(r'RT[\s]+', '', text)        # Menghapus RT
  text = re.sub(r'https?://\S+', '', text)   # Menghapus URL
  text = re.sub(r'[^A-Za-z0-9 ]', '', text)  # Menghapus karakter non-alfanumerik kecuali spasi
  text = re.sub(r'\s+', ' ', text).strip()   # Menghapus spasi berlebih
  return text
df['full_text'] = df['full_text'].apply(clean_twitter_text)
print(f"\nShape setelah Cleaning: {df.shape}")
print("DataFrame setelah Cleaning:")
print(df.head())


Shape setelah Cleaning: (5246, 2)
DataFrame setelah Cleaning:
                                         created_at  \
conversation_id_str                                   
1858070162362204643  Sun Nov 17 08:50:22 +0000 2024   
1846529554657239219  Wed Oct 16 12:32:07 +0000 2024   
1467691301155979267  Wed Oct 09 13:58:44 +0000 2024   
1836014724523458593  Tue Sep 17 12:09:56 +0000 2024   
1835164951125917967  Sun Sep 15 03:53:14 +0000 2024   

                                                             full_text  
conversation_id_str                                                     
1858070162362204643            lucu banget mahasiswa pencinta alam ini  
1846529554657239219  bejat mahasiswa pencinta alam di jambi genjot ...  
1467691301155979267  ada perkoempoelan pentjinta alam yang lahir th...  
1836014724523458593  press release sowan ukm ke ukm unit pandu ling...  
1835164951125917967  press release sowan ukm ke ukm unit pandu ling...  


In [13]:
# Filter Token Length
def filter_tokens_by_length(df, column, min_words, max_words):
    words_count = df[column].astype(str).apply(lambda x: len(x.split()))
    mask = (words_count >= min_words) & (words_count <= max_words)
    filtered_df = df[mask]
    return filtered_df
min_words = 3
max_words = 5500
df = filter_tokens_by_length(df, 'full_text', min_words, max_words)
print(f"\nShape setelah Filter Token Length: {df.shape}")
print("DataFrame setelah Filter Token Length:")
print(df.head())


Shape setelah Filter Token Length: (5170, 2)
DataFrame setelah Filter Token Length:
                                         created_at  \
conversation_id_str                                   
1858070162362204643  Sun Nov 17 08:50:22 +0000 2024   
1846529554657239219  Wed Oct 16 12:32:07 +0000 2024   
1467691301155979267  Wed Oct 09 13:58:44 +0000 2024   
1836014724523458593  Tue Sep 17 12:09:56 +0000 2024   
1835164951125917967  Sun Sep 15 03:53:14 +0000 2024   

                                                             full_text  
conversation_id_str                                                     
1858070162362204643            lucu banget mahasiswa pencinta alam ini  
1846529554657239219  bejat mahasiswa pencinta alam di jambi genjot ...  
1467691301155979267  ada perkoempoelan pentjinta alam yang lahir th...  
1836014724523458593  press release sowan ukm ke ukm unit pandu ling...  
1835164951125917967  press release sowan ukm ke ukm unit pandu ling...  


In [14]:
# Normalisasi Data
norm = {'kekekekegiatanananan': 'kegiatan', 'kamucu': 'kamu', 'jutidak': 'tidak', 'lingkung': 'lingkungan',
    'berberberbersama': 'bersama', 'press release': 'siaran pers', 'ukamu': 'ukm',
    'kekekekegiatanananan': 'kegiatan', 'jutidak': 'tidak', 'lingkung': 'lingkungan',
    'berberberbersama': 'bersama', 'press release': 'siaran pers', 'ukamu': 'ukm',
    'giat': 'kegiatan', 'luring': 'luring', 'sekre': 'sekretariat',
    'mapala': 'mahasiswa pencinta alam', 'kocak': 'lucu', 'enak': 'enak',
    'bilang': 'bilang', 'sowan': 'mengunjungi', 'buka': 'membuka',
    'sampai': 'hingga', 'pukul': 'jam', 'sd': 'sampai', 'kapan': 'kapan',
    'mau': 'ingin', 'join': 'bergabung', 'sama': 'bersama', 'bisa': 'dapat',
    'bantu': 'membantu', 'mikir2': 'mikir', 'perkoempoelan': 'perkumpulan',
    'genjot': 'menyerang', 'wkwkw': 'hahaha', 'th': 'tahun', 'bejat': 'rusak',
    'pentjinta': 'pencinta', 'siaaapp': 'siap', 'okaaay': 'oke', 'udh': 'sudah',
    'ga': 'tidak', 'gaskeun': 'ayo', 'wowww': 'wow', 'haaayyuukkk': 'ayo',
    'yg': 'yang', 'wkwk': '', 'min': 'kak', 'malem': 'malam', 'malem2': 'malam',
    'sm': 'sama', 'dy': 'dia', 'lg': 'lagi', 'skrg': 'sekarang', 'ddpn': 'didepan',
    'makasi': 'makasih', 'pertamaz': 'pertamax', 'jg': 'juga', 'donk': 'dong',
    'ikutann': 'ikutan', 'banyakk': 'banyak', 'twt': 'tweet', 'mantaap': 'mantap',
    'juarak': 'juara', 'daridulu': 'dari dulu', 'siapp': 'siap', 'gamau': 'tidak mau',
    'sll': 'selalu', 'qu': 'aku', 'krn': 'karena', 'irii': 'iri', 'muluu': 'terus',
    'mada': 'masa', 'jgn': 'jangan', 'muluuu': 'terus', 'ntar': 'nanti',
    'awtnya': 'awetnya', 'gg': 'keren', 'kerennn': 'keren', 'bisaa': 'bisa',
    'gaaa': 'tidak', 'nyampe': 'sampai', 'lu': 'kamu', 'ikhlaaasss': 'ikhlas',
    'gak': 'tidak', 'klo': 'kalo', 'kyk': 'seperti', 'sbg': 'sebagai',
    'giat': 'kegiatan', 'luring': 'luring', 'sekre': 'sekretariat',
    'mapala': 'mahasiswa pencinta alam', 'kocak': 'lucu', 'enak': 'enak',
    'bilang': 'bilang', 'sowan': 'mengunjungi', 'buka': 'membuka',
    'sampai': 'hingga', 'pukul': 'jam', 'sd': 'sampai', 'kapan': 'kapan',
    'mau': 'ingin', 'join': 'bergabung', 'sama': 'bersama', 'bisa': 'dapat',
    'bantu': 'membantu', 'mikir2': 'mikir', 'perkoempoelan': 'perkumpulan',
    'genjot': 'menyerang', 'wkwkw': 'hahaha', 'th': 'tahun', 'bejat': 'rusak',
    'pentjinta': 'pencinta', 'siaaapp': 'siap', 'okaaay': 'oke', 'udh': 'sudah',
    'ga': 'tidak', 'gaskeun': 'ayo', 'wowww': 'wow', 'haaayyuukkk': 'ayo',
    'yg': 'yang', 'wkwk': '', 'min': 'kak', 'malem': 'malam', 'malem2': 'malam',
    'sm': 'sama', 'dy': 'dia', 'lg': 'lagi', 'skrg': 'sekarang', 'ddpn': 'didepan',
    'makasi': 'makasih', 'pertamaz': 'pertamax', 'jg': 'juga', 'donk': 'dong',
    'ikutann': 'ikutan', 'banyakk': 'banyak', 'twt': 'tweet', 'mantaap': 'mantap',
    'juarak': 'juara', 'daridulu': 'dari dulu', 'siapp': 'siap', 'gamau': 'tidak mau',
    'sll': 'selalu', 'qu': 'aku', 'krn': 'karena', 'irii': 'iri', 'muluu': 'terus',
    'mada': 'masa', 'jgn': 'jangan', 'muluuu': 'terus', 'ntar': 'nanti',
    'awtnya': 'awetnya', 'gg': 'keren', 'kerennn': 'keren', 'bisaa': 'bisa',
    'gaaa': 'tidak', 'nyampe': 'sampai', 'lu': 'kamu', 'ikhlaaasss': 'ikhlas',
    'gak': 'tidak', 'klo': 'kalo', 'kyk': 'seperti', 'sbg': 'sebagai'}
def normalisasi(str_text):
  for i in norm:
    str_text = str_text.replace(i, norm[i])
  return str_text
df['full_text'] = df['full_text'].apply(lambda x: normalisasi(x))
print("\nDataFrame setelah Normalisasi:")
print(df.head())


DataFrame setelah Normalisasi:
                                         created_at  \
conversation_id_str                                   
1858070162362204643  Sun Nov 17 08:50:22 +0000 2024   
1846529554657239219  Wed Oct 16 12:32:07 +0000 2024   
1467691301155979267  Wed Oct 09 13:58:44 +0000 2024   
1836014724523458593  Tue Sep 17 12:09:56 +0000 2024   
1835164951125917967  Sun Sep 15 03:53:14 +0000 2024   

                                                             full_text  
conversation_id_str                                                     
1858070162362204643          kamucu banget mahasiswa pencinta alam ini  
1846529554657239219  rusak mahasiswa pencinta alam di jambi menyera...  
1467691301155979267  ada perkumpulan pencinta alam yang lahir tahun...  
1836014724523458593  siaran pers mengunjungi ukm ke ukm unit pandu ...  
1835164951125917967  siaran pers mengunjungi ukm ke ukm unit pandu ...  


In [16]:
# Stopwords Removal
more_stop_words = ['anjayyy', 'gtgtgt']
stop_words = StopWordRemoverFactory().get_stop_words()
stop_words.extend(more_stop_words)
new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(new_array)
def stopword(str_text):
  str_text = stop_words_remover_new.remove(str_text)
  return str_text
df['full_text'] = df['full_text'].apply(lambda x: stopword(x))
print("\nDataFrame setelah Stopwords Removal:")
print(df.head())


DataFrame setelah Stopwords Removal:
                                         created_at  \
conversation_id_str                                   
1858070162362204643  Sun Nov 17 08:50:22 +0000 2024   
1846529554657239219  Wed Oct 16 12:32:07 +0000 2024   
1467691301155979267  Wed Oct 09 13:58:44 +0000 2024   
1836014724523458593  Tue Sep 17 12:09:56 +0000 2024   
1835164951125917967  Sun Sep 15 03:53:14 +0000 2024   

                                                             full_text  
conversation_id_str                                                     
1858070162362204643              kamucu banget mahasiswa pencinta alam  
1846529554657239219  rusak mahasiswa pencinta alam jambi menyerang ...  
1467691301155979267  perkumpulan pencinta alam lahir tahun 1953 did...  
1836014724523458593  siaran pers mengunjungi ukm ukm unit pandu lin...  
1835164951125917967  siaran pers mengunjungi ukm ukm unit pandu lin...  


In [17]:
# Tokenisasi
tokenized_for_stemming = df['full_text'].apply(lambda x: x.split())
print("\nTokenisasi (sebelum Stemming):")
print(tokenized_for_stemming.head())


Tokenisasi (sebelum Stemming):
conversation_id_str
1858070162362204643          [kamucu, banget, mahasiswa, pencinta, alam]
1846529554657239219    [rusak, mahasiswa, pencinta, alam, jambi, meny...
1467691301155979267    [perkumpulan, pencinta, alam, lahir, tahun, 19...
1836014724523458593    [siaran, pers, mengunjungi, ukm, ukm, unit, pa...
1835164951125917967    [siaran, pers, mengunjungi, ukm, ukm, unit, pa...
Name: full_text, dtype: object


In [19]:
# Stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()
def stemming_text(list_of_tokens):
  stemmed_tokens = [stemmer.stem(w) for w in list_of_tokens]
  return " ".join(stemmed_tokens)
df['full_text'] = tokenized_for_stemming.apply(stemming_text)
print("\nDataFrame setelah Stemming:")
df.to_csv('/content/stemmingg.csv', index=False)
print(df.head())
print(f"Final DataFrame Shape setelah preprocessing: {df.shape}")

KeyboardInterrupt: 

In [27]:
print("\nDataframe setelah preprocessing disimpan ke /content/stemmingg.csv")
print(f"Final DataFrame Shape setelah preprocessing: {df.shape}")


Dataframe setelah preprocessing disimpan ke /content/stemmingg.csv
Final DataFrame Shape setelah preprocessing: (5170, 2)


In [ ]:
# Labeling
print("\nLabeling")
df = pd.read_csv('/content/stemming.csv')
classifier = pipeline("sentiment-analysis", model="w11wo/indonesian-roberta-base-sentiment-classifier")
def prediksi_sentimen(teks):
    if not isinstance(teks, str) or not teks.strip():
        return 'neutral'
    try:
        hasil = classifier(teks)
        return hasil[0]['label']
    except Exception as e:
        print(f"Error predicting sentiment for text: '{teks[:50]}...' - {e}")
        return 'neutral'
print("\nMemulai proses Labeling")
df['sentimen'] = df['full_text'].apply(prediksi_sentimen)
print("Labeling selesai.")
print("\nDataframe dengan sentimen disimpan ke /content/labelinggg.csv")


Labeling


Device set to use cpu
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Memulai proses Labeling


In [ ]:
print("\n/content/labelinggg.csv")

In [ ]:
sentimen_counts = df.sentimen.value_counts()
print("\nDistribusi Sentimen:")
print(sentimen_counts)
print(f"\nFinal DataFrame Shape: {df.shape}")

In [ ]:
# Visualisasi
print("\nVisualisasi")
sns.set_palette("pastel")
plt.figure(figsize=(8, 6))
sns.countplot(x='sentimen', data=df)
plt.title('Analisis Sentimen Publik di X Terhadap Peran Mahasiswa Pencinta Alam dalam Pelestarian Lingkungan')
plt.xlabel('Sentimen')
plt.ylabel('Jumlah')
plt.show()

In [ ]:
data_positif = df[df['sentimen'] == 'positive']
data_negatif = df[df['sentimen'] == 'negative']
data_netral = df[df['sentimen'] == 'neutral']

In [ ]:
# Word Cloud Positif
if not data_positif.empty:
    all_text_s1 = ' '.join(word for word in data_positif["full_text"].dropna())
    if all_text_s1:
        wordcloud = WordCloud(colormap='Blues', width=1000, height=1000, mode="RGBA", background_color='white').generate(all_text_s1)
        plt.figure(figsize=(9, 6))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.title("Visualisasi Kata Positif")
        plt.margins(x=0, y=0)
        plt.show()

In [ ]:
# Word Cloud Negatif
if not data_negatif.empty:
    all_text_s0 = ' '.join(word for word in data_negatif["full_text"].dropna())
    if all_text_s0:
        wordcloud = WordCloud(colormap='Reds', width=1000, height=1000, mode='RGBA', background_color='white').generate(all_text_s0)
        plt.figure(figsize=(9, 6))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title("Visualisasi Kata Negatif")
        plt.margins(x=0, y=0)
        plt.show()

In [ ]:
# Word Cloud Netral
if not data_netral.empty:
    all_text_s2 = ' '.join(word for word in data_netral["full_text"].dropna())
    if all_text_s2:
        wordcloud = WordCloud(colormap='Greens', width=1000, height=1000, mode='RGBA', background_color='white').generate(all_text_s2)
        plt.figure(figsize=(9, 6))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title("Visualisasi Kata Netral")
        plt.margins(x=0, y=0)
        plt.show()

In [ ]:
# Pembagian Data untuk Pelatihan
print("\nPembagian Data untuk Pelatihan")
x = df.full_text
y = df.sentimen

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
print('\nBanyak data x_train :',len(x_train))
print('Banyak data x_test  :',len(x_test))
print('Banyak data y_train :',len(y_train))
print('Banyak data y_test  :',len(y_test))

In [ ]:
print("\nMENAMPILKAN DATA TEST DAN TRAIN")
print("\ndata pertama dari x_train (teks yang sudah diproses):")
print(x_train.head().to_string())
print("\ndata pertama dari x_test (teks yang sudah diproses):")
print(x_test.head().to_string())
print("\ndata pertama dari y_train (label sentimen):")
print(y_train.head())
print("\ndata pertama dari y_test (label sentimen):")
print(y_test.head())

In [ ]:
print(f"\nUkuran x_train: {x_train.shape}")
print(f"Ukuran x_test: {x_test.shape}")
print(f"Ukuran y_train: {y_train.shape}")
print(f"Ukuran y_test: {y_test.shape}")

In [ ]:
# Pelatihan dan Evaluasi Model Naive Bayes dengan Hyperparameter Tuning
print("pelatihan dan Evaluasi Model Naive Bayes")
# Definisi Pipeline untuk Naive Bayes
pipeline_nb = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', MultinomialNB())
])

In [ ]:
# Parameter Grid untuk GridSearchCV
# Eksperimen dengan ngram_range dan max_features untuk TfidfVectorizer
# Eksperimen dengan alpha untuk MultinomialNB
param_grid = {
    'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)], # Coba unigram, bigram, trigram
    'vectorizer__max_features': [5000, 10000, 15000], # Jumlah fitur TF-IDF
    'classifier__alpha': [0.01, 0.1, 0.5, 1.0, 2.0] # Smoothing parameter untuk Naive Bayes
}

In [ ]:
# Inisialisasi GridSearchCV
# cv=5 berarti 5-fold cross-validation
# n_jobs=-1 berarti menggunakan semua core CPU yang tersedia
grid_search_nb = GridSearchCV(pipeline_nb, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
print("\nMemulai GridSearchCV untuk Naive Bayes (ini mungkin memakan waktu)...")
grid_search_nb.fit(x_train, y_train)
print("\nGridSearchCV selesai.")
print(f"Best parameters found: {grid_search_nb.best_params_}")
print(f"Best cross-validation accuracy: {grid_search_nb.best_score_:.4f}")

In [ ]:
print("\ndata pertama dari x_train (teks yang sudah diproses):")
print(x_train.head().to_string())

In [ ]:
print("\ndata pertama dari x_test (teks yang sudah diproses):")
print(x_test.head().to_string())

In [ ]:
print("\ndata pertama dari y_train (label sentimen):")
print(y_train.head())

In [ ]:
print("\ndata pertama dari y_test (label sentimen):")
print(y_test.head())

In [ ]:
print(f"\nUkuran x_train: {x_train.shape}")
print(f"Ukuran x_test: {x_test.shape}")
print(f"Ukuran y_train: {y_train.shape}")
print(f"Ukuran y_test: {y_test.shape}")

In [ ]:
# Pelatihan Model Naive Bayes dengan TF-IDF
print("\nPelatihan Model Naive Bayes dengan TF-IDF")

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=5170)
naive_bayes_classifier = MultinomialNB()
model_naive_bayes = Pipeline([
    ('vectorizer', tfidf_vectorizer),
    ('classifier', naive_bayes_classifier)
])

In [ ]:
# Latih model Naive Bayes
model_naive_bayes.fit(x_train, y_train)
print("Model Naive Bayes berhasil dilatih.")

In [ ]:
print("\nMENAMPILKAN REPRESENTASI TF-IDF DAN PROBABILITAS")
trained_tfidf_vectorizer = model_naive_bayes.named_steps['vectorizer']
X_train_tfidf_transformed = trained_tfidf_vectorizer.transform(x_train)
X_test_tfidf_transformed = trained_tfidf_vectorizer.transform(x_test)
print(f"\nShape TF-IDF dari x_train: {X_train_tfidf_transformed.shape}")
print(f"Shape TF-IDF dari x_test: {X_test_tfidf_transformed.shape}")


In [ ]:
print("\nbaris pertama dari matriks TF-IDF x_train (sparse format):")
print(X_train_tfidf_transformed[:10])

In [ ]:
print("\nJumlah fitur (kata unik) yang dipelajari oleh TF-IDF Vectorizer:", len(trained_tfidf_vectorizer.get_feature_names_out()))
print("\nBeberapa fitur (kata) pertama yang dipelajari oleh TF-IDF Vectorizer:")
print(trained_tfidf_vectorizer.get_feature_names_out()[:20])

In [ ]:
# Untuk melihat TF-IDF dari sebuah teks
if not x_train.empty:
    sample_text_train = x_train.iloc[0]
    sample_tfidf_train = trained_tfidf_vectorizer.transform([sample_text_train])
    print(f"\nTF-IDF untuk contoh teks pertama dari x_train ('{sample_text_train[:50]}...'):")

    print(sample_tfidf_train.toarray())

    feature_names = trained_tfidf_vectorizer.get_feature_names_out()
    print("Fitur dengan bobot non-nol untuk contoh teks ini:")

    non_zero_cols = sample_tfidf_train.nonzero()[1]
    for col_idx in non_zero_cols:
        print(f"  {feature_names[col_idx]}: {sample_tfidf_train[0, col_idx]:.4f}")


In [ ]:
print("\n--- Probabilitas Prediksi")
y_proba_nb = model_naive_bayes.predict_proba(x_test)
print("\nProbabilitas prediksi untuk 10 sampel pertama dari x_test:")
proba_df = pd.DataFrame(y_proba_nb[:5], columns=model_naive_bayes.classes_)
print(proba_df)

In [ ]:
print("\nContoh probabilitas untuk satu teks dari x_test:")
if not x_test.empty:
    sample_text_test = x_test.iloc[0]
    sample_proba = model_naive_bayes.predict_proba([sample_text_test])
    print(f"Teks: '{sample_text_test[:50]}...'")
    print(f"Probabilitas: {sample_proba[0]}")
    # Menampilkan probabilitas dengan label kelas
    for i, class_label in enumerate(model_naive_bayes.classes_):
        print(f"  {class_label}: {sample_proba[0][i]:.4f}")
    print(f"Prediksi kelas: {model_naive_bayes.predict([sample_text_test])[0]}")

In [ ]:
# MENAMPILKAN TF-IDF DAN PROBABILITAS
#print("\nEvaluasi Model Naive Bayes")
y_pred_nb = model_naive_bayes.predict(x_test)

In [ ]:
# Confusion Matrix
from sklearn.metrics import ConfusionMatrixDisplay
cm_nb = confusion_matrix(y_test, y_pred_nb)
disp_nb = ConfusionMatrixDisplay(confusion_matrix=cm_nb, display_labels=model_naive_bayes.classes_)
disp_nb.plot(cmap='Blues')
plt.title('Confusion Matrix Naive Bayes')
plt.show()

In [ ]:
# Classification Report
matrix_nb = classification_report(y_test, y_pred_nb)
print('Classification Report (Naive Bayes):\n', matrix_nb)

In [ ]:
# Informasi Internal Model Naive Bayes
print("\nINFORMASI INTERNAL MODEL NAIVE BAYES")
trained_vectorizer = model_naive_bayes.named_steps['vectorizer']
trained_classifier = model_naive_bayes.named_steps['classifier']

In [ ]:
vocabulary = trained_vectorizer.vocabulary_
print("\nUkuran Vocabulary:", len(vocabulary))

In [ ]:
X_train_tfidf_transformed_for_info = trained_vectorizer.transform(x_train) # Re-transform for info if needed
print(f"\nShape Matriks TF-IDF (x_train): {X_train_tfidf_transformed_for_info.shape}")

In [ ]:
class_labels = trained_classifier.classes_
print("\nNama Kelas (Sentimen):", class_labels)


In [ ]:
log_prior_classes = trained_classifier.class_log_prior_
print("\nLog Probabilitas Prior Kelas:")
for i, label in enumerate(class_labels):
    print(f"  Log P({label}): {log_prior_classes[i]:.4f}")

In [ ]:
log_likelihood_features = trained_classifier.feature_log_prob_
print(f"\nShape Log Probabilitas Likelihood Fitur: {log_likelihood_features.shape}")

In [ ]:
print("\n10 Kata Teratas dengan Probabilitas Tertinggi per Kelas:")
feature_names = trained_vectorizer.get_feature_names_out()
for i, class_label in enumerate(class_labels):
    top_10_indices = log_likelihood_features[i, :].argsort()[-10:][::-1]
    top_10_words = [feature_names[idx] for idx in top_10_indices]
    top_10_probs = [f"{log_likelihood_features[i, idx]:.4f}" for idx in top_10_indices]
    print(f"  Kelas '{class_label}':")
    for j in range(10):
        print(f"    - {top_10_words[j]} (Log Prob: {top_10_probs[j]})")

In [ ]:
# Pengujian Model Naïve Bayes
print("\nPengujian Model Naive Bayes")
def classify_text_with_naive_bayes(input_text):

    processed_text = input_text.lower()
    processed_text = clean_twitter_text(processed_text)
    processed_text = normalisasi(processed_text)
    processed_text = stopword(processed_text) # Corrected function name
    processed_text = stemming_text(processed_text) # Pastikan ini mengembalikan string
    # Prediksi menggunakan pipeline Naive Bayes
    prediction = model_naive_bayes.predict([processed_text])
    # Probabilitas prediksi
    prediction_proba = model_naive_bayes.predict_proba([processed_text])
    return prediction[0], prediction_proba[0]

input_text_user = input("\nMasukkan teks yang ingin diklasifikasikan oleh Naive Bayes: ")
results_nb_class, results_nb_proba = classify_text_with_naive_bayes(input_text_user)
print("Input teks :", input_text_user)
print(f"Hasil Klasifikasi (Multinomial Naive Bayes): {results_nb_class}")
print("Probabilitas untuk setiap kelas:")
for i, class_label in enumerate(model_naive_bayes.classes_):
    print(f"  {class_label}: {results_nb_proba[i]:.4f}")